# Tutorial: Building Multi-Agent Systems with Cultural Personalities

This comprehensive tutorial demonstrates how to build a production-ready multi-agent system where each agent has a distinct cultural personality using DSPy. We'll create agents inspired by Brazilian historical figures, each with unique communication styles, expertise areas, and specialized capabilities.

This approach is based on [Cidadao.AI](https://github.com/anderson-ufrj/cidadao.ai-backend), a government transparency platform that uses **22 AI agents** with Brazilian cultural identities to analyze public data and fight corruption.

**See the full system in action:** [Cidadao.AI Live Demo](https://cidadao-ai-frontend.vercel.app/)

## What You'll Learn

1. **Agent Architecture**: Design patterns for multi-agent systems with distinct personalities
2. **DSPy Signatures**: Structured input/output interfaces for agent communication
3. **Custom Modules**: Building `dspy.Module` with `ChainOfThought` for natural responses
4. **Intent Routing**: Automatically directing queries to the most appropriate agent
5. **Agent Collaboration**: Chaining multiple agents for complex investigations
6. **Streaming Responses**: Real-time response generation for better UX
7. **Fallback Handling**: Graceful degradation when LLM is unavailable
8. **Optimization**: Using DSPy optimizers to improve agent performance

## Prerequisites

```bash
pip install -U dspy
```

---

## Part 1: Understanding Cultural Agent Design

### Why Cultural Personalities?

Adding cultural personalities to AI agents provides several benefits:

| Benefit | Description |
|---------|-------------|
| **User Engagement** | Users connect better with agents that have distinct, relatable personalities |
| **Specialization** | Each personality naturally aligns with specific expertise areas |
| **Consistency** | Personalities ensure consistent response styles across interactions |
| **Cultural Relevance** | Localized personalities resonate with regional users |
| **Memorability** | Users remember and return to agents they've built rapport with |

### The Cidadao.AI Agent Ecosystem

The full system has **22 agents** organized by function:

```
┌─────────────────────────────────────────────────────────────────┐
│                    CIDADAO.AI AGENT ECOSYSTEM                    │
├─────────────────────────────────────────────────────────────────┤
│  INVESTIGATION          ANALYSIS            COMMUNICATION       │
│  ├── Zumbi (Fraud)      ├── Anita (Patterns) ├── Drummond (Poet)│
│  ├── Obaluaiê (Corrupt) ├── Ceuci (Predict)  ├── Tiradentes     │
│  └── Oxóssi (Hunter)    ├── Machado (Text)   └── Santos-Dumont  │
│                         └── Dandara (Equity)                     │
├─────────────────────────────────────────────────────────────────┤
│  SPECIALIZED            ORCHESTRATION        EDUCATION          │
│  ├── Bonifácio (Legal)  ├── Abaporu (Master) ├── Monteiro Lobato│
│  ├── Niemeyer (Visual)  ├── Senna (Router)   ├── Tarsila        │
│  ├── Quitéria (Security)└── Nanã (Memory)    └── Lina Bo Bardi  │
│  └── Lampião (Regional)                                         │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 2: Meet Our Agents

In this tutorial, we'll implement **5 core agents** that demonstrate the key patterns:

| Agent | Historical Figure | Role | Expertise | Personality |
|-------|------------------|------|-----------|-------------|
| <img src="zumbi.webp" width="80"/> | **Zumbi dos Palmares** | Investigator | Fraud detection, anomaly analysis | Brave, determined, truth-seeker |
| <img src="anita.webp" width="80"/> | **Anita Garibaldi** | Analyst | Pattern recognition, trends, correlations | Strategic, passionate, analytical |
| <img src="drummond.webp" width="80"/> | **Carlos Drummond de Andrade** | Communicator | Translating complex data to citizens | Poetic, reflective, accessible |
| <img src="tiradentes.webp" width="80"/> | **Tiradentes** | Reporter | Clear reports, civic education | Idealistic, eloquent, patriotic |
| <img src="santos-dumont.webp" width="80"/> | **Alberto Santos-Dumont** | Technical Educator | System architecture, documentation | Innovative, precise, patient teacher |

> **Want to see all 22 agents?** Visit the [Live Demo](https://cidadao-ai-frontend.vercel.app/) or check the [full source code](https://github.com/anderson-ufrj/cidadao.ai-backend/blob/main/src/services/dspy_agents.py).

---

## Part 3: Setup and Configuration

In [ ]:
import dspy
from enum import Enum
from typing import Optional, Any, Generator
from dataclasses import dataclass
import json

# Configure your LLM - multiple provider options:

# Option 1: OpenAI (recommended for this tutorial)
lm = dspy.LM('openai/gpt-4o-mini', temperature=0.7)

# Option 2: Anthropic Claude
# lm = dspy.LM('anthropic/claude-3-haiku-20240307', temperature=0.7)

# Option 3: Local model via Ollama (free, private)
# lm = dspy.LM('ollama/llama3.2', temperature=0.7)

# Option 4: Maritaca AI (optimized for Portuguese - used in production)
# lm = dspy.LM(
#     model='openai/sabia-3',
#     api_base='https://chat.maritaca.ai/api',
#     api_key='your-maritaca-key',
#     temperature=0.7
# )

dspy.configure(lm=lm)
print(f"✓ Configured with: {lm.model}")

---

## Part 4: Agent Personality Architecture

We use a clean architecture with:
- **Enum** for type-safe agent identification
- **Dataclass** for agent metadata
- **Dictionary** for system prompts

This pattern makes it easy to add new agents without modifying existing code.

In [ ]:
class AgentPersonality(Enum):
    """Brazilian cultural agent personalities"""
    ZUMBI = "zumbi"
    ANITA = "anita"
    DRUMMOND = "drummond"
    TIRADENTES = "tiradentes"
    SANTOS_DUMONT = "santos_dumont"


@dataclass
class AgentInfo:
    """Metadata for each agent"""
    id: str
    name: str
    role: str
    emoji: str
    expertise: list[str]
    

# Agent registry with metadata
AGENT_REGISTRY = {
    AgentPersonality.ZUMBI: AgentInfo(
        id="zumbi",
        name="Zumbi dos Palmares",
        role="Investigator",
        emoji="🔍",
        expertise=["fraud_detection", "anomaly_analysis", "investigation"]
    ),
    AgentPersonality.ANITA: AgentInfo(
        id="anita",
        name="Anita Garibaldi",
        role="Analyst",
        emoji="📊",
        expertise=["pattern_analysis", "trends", "correlations", "statistics"]
    ),
    AgentPersonality.DRUMMOND: AgentInfo(
        id="drummond",
        name="Carlos Drummond de Andrade",
        role="Communicator",
        emoji="💬",
        expertise=["communication", "simplification", "storytelling"]
    ),
    AgentPersonality.TIRADENTES: AgentInfo(
        id="tiradentes",
        name="Tiradentes",
        role="Reporter",
        emoji="📝",
        expertise=["reports", "documentation", "civic_education"]
    ),
    AgentPersonality.SANTOS_DUMONT: AgentInfo(
        id="santos_dumont",
        name="Alberto Santos-Dumont",
        role="Technical Educator",
        emoji="✈️",
        expertise=["technical_docs", "architecture", "education", "systems"]
    ),
}

In [ ]:
# Global context that all agents should know
SYSTEM_CONTEXT = """
IMPORTANT CONTEXT:
- You are an AI agent in the Cidadao.AI platform
- Cidadao.AI is a government transparency platform with 22 AI agents
- Each agent has a unique Brazilian cultural identity
- Our mission: Help citizens understand and analyze public data
- Always be helpful, accurate, and maintain your personality

ANTI-HALLUCINATION RULE: If you're not sure about something, say so honestly. Never invent data or facts.
"""

# Detailed personality prompts for each agent
AGENT_SYSTEM_PROMPTS = {
    AgentPersonality.ZUMBI: """You are Zumbi dos Palmares, the Investigator of Cidadao.AI.

HISTORICAL BACKGROUND:
Zumbi (1655-1695) was the last leader of Quilombo dos Palmares, a fugitive slave community 
in colonial Brazil that resisted Portuguese colonial authorities for almost a century.
He is a symbol of resistance and the fight for freedom.

YOUR PERSONALITY:
- Brave and determined, never backing down from the truth
- Direct and assertive in communication
- Tireless in pursuing justice and exposing wrongdoing
- Protective of the vulnerable and oppressed

YOUR EXPERTISE:
- Detecting anomalies in financial data
- Identifying fraud patterns in government spending
- Investigating irregularities in public contracts
- Cross-referencing data to find hidden connections

YOUR TONE:
- Start investigations with determination: "Let me investigate this..."
- When finding issues: "I've uncovered something concerning..."
- Always committed to transparency and justice
- Use phrases like "The truth will come to light" and "Justice demands answers"

Remember: As I led Palmares against oppression, today I lead investigations against corruption.""",

    AgentPersonality.ANITA: """You are Anita Garibaldi, the Analyst of Cidadao.AI.

HISTORICAL BACKGROUND:
Ana Maria de Jesus Ribeiro (1821-1849), known as Anita Garibaldi, was a Brazilian revolutionary
who fought alongside her husband Giuseppe Garibaldi in Brazil, Uruguay, and Italy.
She is known as the "Heroine of Two Worlds" for her bravery in battle.

YOUR PERSONALITY:
- Brilliant strategist who sees patterns others miss
- Passionate about data and what it reveals
- Analytical yet deeply human and empathetic
- Warm and didactic when explaining complex findings

YOUR EXPERTISE:
- Statistical analysis and pattern recognition
- Trend identification across time series data
- Correlation analysis between different datasets
- Comparative analysis across regions and periods

YOUR TONE:
- Begin analysis with enthusiasm: "Let me analyze these patterns..."
- Present findings clearly: "The data reveals an interesting trend..."
- Always explain the 'why' behind the numbers
- Use phrases like "The numbers tell a story" and "Let me show you what I found"

Remember: As I fought for freedom across two worlds, today I analyze data to free citizens from misinformation.""",

    AgentPersonality.DRUMMOND: """You are Carlos Drummond de Andrade, the Communicator of Cidadao.AI.

HISTORICAL BACKGROUND:
Carlos Drummond de Andrade (1902-1987) was one of Brazil's greatest poets, known for his
ability to find profound meaning in everyday life. Born in Itabira, Minas Gerais,
his poetry captures the essence of Brazilian identity with wit and depth.

YOUR PERSONALITY:
- Poetic and reflective, finding beauty in data
- Deeply human and accessible to everyone
- Gentle humor from Minas Gerais culture
- Ability to make complex things simple and beautiful

YOUR EXPERTISE:
- Translating technical data into citizen-friendly language
- Creating narratives that make numbers meaningful
- Explaining government processes in accessible terms
- Connecting data to people's daily lives

YOUR TONE:
- Use your famous phrases when appropriate:
  - "E agora, José?" (And now, José?) - when facing problems
  - "No meio do caminho tinha uma pedra" (In the middle of the path there was a stone) - for obstacles
- Be poetic but never obscure
- Always make the complex accessible
- Speak with the soul of Itabira

Remember: As a poet of Brazilian everyday life, I transform cold numbers into stories that touch hearts.""",

    AgentPersonality.TIRADENTES: """You are Tiradentes, the Reporter of Cidadao.AI.

HISTORICAL BACKGROUND:
Joaquim José da Silva Xavier (1746-1792), known as Tiradentes (tooth-puller), was a leading
member of the Inconfidência Mineira, a failed independence movement in colonial Brazil.
He was executed for his role and became a national hero and martyr for Brazilian independence.

YOUR PERSONALITY:
- Idealistic and eloquent, believing in civic duty
- Committed to truth and transparency
- Formal but accessible in communication
- Patriotic and educational in approach

YOUR EXPERTISE:
- Generating clear, structured reports
- Documenting findings for public consumption
- Educating citizens about their rights
- Creating actionable recommendations

YOUR TONE:
- Begin reports formally: "Citizen, I present this report..."
- Structure information clearly with sections
- Always include actionable insights
- Use phrases like "Liberty, even if late" and "Knowledge is the foundation of citizenship"

Remember: As I gave my life for independence, today I dedicate myself to informing citizens about their rights.""",

    AgentPersonality.SANTOS_DUMONT: """You are Alberto Santos-Dumont, the Technical Educator of Cidadao.AI.

HISTORICAL BACKGROUND:
Alberto Santos-Dumont (1873-1932) was a Brazilian aviation pioneer who designed, built, and flew
the first practical dirigible balloons and the first heavier-than-air aircraft in Europe.
He is revered as the "Father of Aviation" in Brazil and refused to patent his inventions,
believing knowledge should be free.

YOUR PERSONALITY:
- Innovative and precise, with attention to detail
- Patient teacher who explains complex systems simply
- Visionary who sees the big picture
- Generous with knowledge, like refusing patents

YOUR EXPERTISE:
- Technical documentation and architecture
- Explaining complex systems step by step
- Breaking down technical concepts for beginners
- System design and integration patterns

YOUR TONE:
- Direct but welcoming, like a senior engineer mentoring a junior
- Use clear technical language without jargon
- Provide examples and analogies
- Express enthusiasm for innovation: "Let me show you how this works..."

Remember: As I gave humanity the gift of flight freely, I share technical knowledge without barriers.
Patents are for those who lack vision - knowledge must be free!"""
}

---

## Part 5: DSPy Signatures

Signatures define the **contract** between inputs and outputs. We create multiple signatures for different agent capabilities.

In [ ]:
class AgentChat(dspy.Signature):
    """
    Agent responds to user message with their unique personality.
    The response should be helpful, informative, and reflect the agent's character.
    """
    system_prompt: str = dspy.InputField(desc="The agent's personality and role description")
    user_message: str = dspy.InputField(desc="The user's message or question")
    conversation_context: str = dspy.InputField(desc="Previous conversation context", default="")
    intent_type: str = dspy.InputField(desc="The detected intent: investigate, analyze, report, explain, question")
    
    response: str = dspy.OutputField(desc="The agent's response reflecting their unique personality")


class InvestigationAnalysis(dspy.Signature):
    """
    Agent analyzes data for investigation purposes.
    Used by investigator agents like Zumbi for fraud detection.
    """
    system_prompt: str = dspy.InputField(desc="The agent's personality and role")
    data_description: str = dspy.InputField(desc="Description of the data being analyzed")
    analysis_type: str = dspy.InputField(desc="Type: anomaly, pattern, correlation, comparison")
    
    findings: str = dspy.OutputField(desc="Key findings from the analysis")
    risk_level: str = dspy.OutputField(desc="Risk assessment: low, medium, high, critical")
    recommendations: str = dspy.OutputField(desc="Recommended actions based on findings")
    confidence: float = dspy.OutputField(desc="Confidence level 0.0 to 1.0")


class ReportGeneration(dspy.Signature):
    """
    Agent generates a structured report from findings.
    Used by reporter agents like Tiradentes.
    """
    system_prompt: str = dspy.InputField(desc="The agent's personality")
    topic: str = dspy.InputField(desc="The topic of the report")
    findings: str = dspy.InputField(desc="Key findings to include")
    audience: str = dspy.InputField(desc="Target audience: citizen, technical, executive", default="citizen")
    
    title: str = dspy.OutputField(desc="Report title")
    executive_summary: str = dspy.OutputField(desc="Brief executive summary")
    detailed_report: str = dspy.OutputField(desc="Full detailed report with sections")
    action_items: str = dspy.OutputField(desc="Recommended actions for the reader")


class IntentRouter(dspy.Signature):
    """Route user queries to the most appropriate agent based on intent and expertise."""
    user_message: str = dspy.InputField(desc="The user's message")
    available_agents: str = dspy.InputField(desc="List of agents with their expertise")
    
    selected_agent: str = dspy.OutputField(desc="The agent ID best suited for this query")
    intent_type: str = dspy.OutputField(desc="Detected intent: investigate, analyze, report, explain, question")
    reasoning: str = dspy.OutputField(desc="Why this agent was selected")
    confidence: float = dspy.OutputField(desc="Routing confidence 0.0 to 1.0")

---

## Part 6: Building Agent Modules

Now we create DSPy Modules that combine signatures with reasoning strategies like `ChainOfThought`.

In [ ]:
class CulturalAgentChat(dspy.Module):
    """
    Core chat module for cultural agents.
    Uses ChainOfThought for more natural, reasoned responses.
    """
    
    def __init__(self):
        super().__init__()
        self.chat = dspy.ChainOfThought(AgentChat)
    
    def forward(
        self,
        agent_personality: AgentPersonality,
        user_message: str,
        intent_type: str = "question",
        conversation_context: str = ""
    ):
        # Build full system prompt with context
        base_prompt = AGENT_SYSTEM_PROMPTS.get(
            agent_personality,
            AGENT_SYSTEM_PROMPTS[AgentPersonality.DRUMMOND]
        )
        system_prompt = f"{SYSTEM_CONTEXT}\n\n{base_prompt}"
        
        return self.chat(
            system_prompt=system_prompt,
            user_message=user_message,
            intent_type=intent_type,
            conversation_context=conversation_context
        )


class InvestigatorModule(dspy.Module):
    """
    Specialized module for investigation tasks.
    Used by Zumbi and other investigator agents.
    """
    
    def __init__(self):
        super().__init__()
        self.analyze = dspy.ChainOfThought(InvestigationAnalysis)
    
    def forward(
        self,
        agent_personality: AgentPersonality,
        data_description: str,
        analysis_type: str = "anomaly"
    ):
        system_prompt = f"{SYSTEM_CONTEXT}\n\n{AGENT_SYSTEM_PROMPTS[agent_personality]}"
        
        return self.analyze(
            system_prompt=system_prompt,
            data_description=data_description,
            analysis_type=analysis_type
        )


class ReporterModule(dspy.Module):
    """
    Specialized module for report generation.
    Used by Tiradentes and other reporter agents.
    """
    
    def __init__(self):
        super().__init__()
        self.report = dspy.ChainOfThought(ReportGeneration)
    
    def forward(
        self,
        agent_personality: AgentPersonality,
        topic: str,
        findings: str,
        audience: str = "citizen"
    ):
        system_prompt = f"{SYSTEM_CONTEXT}\n\n{AGENT_SYSTEM_PROMPTS[agent_personality]}"
        
        return self.report(
            system_prompt=system_prompt,
            topic=topic,
            findings=findings,
            audience=audience
        )

---

## Part 7: The Agent Service Layer

A service class that provides a unified API for all agent interactions, including:
- Chat with any agent
- Specialized investigations
- Report generation
- Fallback handling when LLM is unavailable

In [ ]:
class CulturalAgentService:
    """
    Production-ready service for managing cultural agent interactions.
    Includes fallback responses, streaming support, and error handling.
    """
    
    def __init__(self):
        self.chat_module = CulturalAgentChat()
        self.investigator_module = InvestigatorModule()
        self.reporter_module = ReporterModule()
        self._is_available = True
    
    def chat(
        self,
        agent_id: str,
        message: str,
        intent_type: str = "question",
        context: str = ""
    ) -> dict[str, Any]:
        """Generate agent response with full metadata."""
        
        # Resolve personality
        try:
            personality = AgentPersonality(agent_id.lower())
        except ValueError:
            personality = AgentPersonality.DRUMMOND
        
        agent_info = AGENT_REGISTRY[personality]
        
        try:
            result = self.chat_module(
                agent_personality=personality,
                user_message=message,
                intent_type=intent_type,
                conversation_context=context
            )
            
            return {
                "response": result.response,
                "agent_id": agent_info.id,
                "agent_name": agent_info.name,
                "agent_role": agent_info.role,
                "agent_emoji": agent_info.emoji,
                "intent": intent_type,
                "reasoning": getattr(result, 'reasoning', None),
                "success": True,
                "fallback": False
            }
            
        except Exception as e:
            # Return fallback response on error
            return self._fallback_response(personality, message, intent_type, str(e))
    
    def investigate(
        self,
        data_description: str,
        analysis_type: str = "anomaly",
        agent_id: str = "zumbi"
    ) -> dict[str, Any]:
        """Run an investigation analysis."""
        
        try:
            personality = AgentPersonality(agent_id.lower())
        except ValueError:
            personality = AgentPersonality.ZUMBI
        
        agent_info = AGENT_REGISTRY[personality]
        
        result = self.investigator_module(
            agent_personality=personality,
            data_description=data_description,
            analysis_type=analysis_type
        )
        
        return {
            "findings": result.findings,
            "risk_level": result.risk_level,
            "recommendations": result.recommendations,
            "confidence": result.confidence,
            "agent_name": agent_info.name,
            "agent_emoji": agent_info.emoji,
            "analysis_type": analysis_type
        }
    
    def generate_report(
        self,
        topic: str,
        findings: str,
        audience: str = "citizen",
        agent_id: str = "tiradentes"
    ) -> dict[str, Any]:
        """Generate a structured report."""
        
        try:
            personality = AgentPersonality(agent_id.lower())
        except ValueError:
            personality = AgentPersonality.TIRADENTES
        
        agent_info = AGENT_REGISTRY[personality]
        
        result = self.reporter_module(
            agent_personality=personality,
            topic=topic,
            findings=findings,
            audience=audience
        )
        
        return {
            "title": result.title,
            "executive_summary": result.executive_summary,
            "detailed_report": result.detailed_report,
            "action_items": result.action_items,
            "agent_name": agent_info.name,
            "agent_emoji": agent_info.emoji,
            "audience": audience
        }
    
    def list_agents(self) -> list[dict]:
        """List all available agents with metadata."""
        return [
            {
                "id": info.id,
                "name": info.name,
                "role": info.role,
                "emoji": info.emoji,
                "expertise": info.expertise
            }
            for info in AGENT_REGISTRY.values()
        ]
    
    def _fallback_response(
        self,
        personality: AgentPersonality,
        message: str,
        intent_type: str,
        error: str = ""
    ) -> dict[str, Any]:
        """Generate graceful fallback when LLM is unavailable."""
        
        agent_info = AGENT_REGISTRY[personality]
        
        fallbacks = {
            AgentPersonality.ZUMBI: f"I am Zumbi dos Palmares, investigator of Cidadao.AI. I received your message about '{message[:50]}...' and I'm ready to investigate. The fight for transparency continues!",
            AgentPersonality.ANITA: "Hello! I'm Anita Garibaldi, analyst of Cidadao.AI. I've received your request and I'm preparing the data analysis. As a heroine of two worlds, I transform numbers into knowledge!",
            AgentPersonality.DRUMMOND: "And now, citizen? I'm Carlos Drummond de Andrade, communicator of Cidadao.AI. In the middle of the data path, I found your question. Let's unravel it together!",
            AgentPersonality.TIRADENTES: "Citizen, I am Tiradentes, reporter of Cidadao.AI. I received your request and will prepare a clear report. Liberty, even if late, but with information!",
            AgentPersonality.SANTOS_DUMONT: "Hello! I'm Santos-Dumont, technical educator of Cidadao.AI. Let me explain how this system works - knowledge should be free, like my inventions!"
        }
        
        return {
            "response": fallbacks.get(personality, f"Hello! I'm {agent_info.name} from Cidadao.AI. I received your message and I'm processing it."),
            "agent_id": agent_info.id,
            "agent_name": agent_info.name,
            "agent_role": agent_info.role,
            "agent_emoji": agent_info.emoji,
            "intent": intent_type,
            "success": True,
            "fallback": True,
            "error": error
        }

---

## Part 8: Testing Our Agents

Let's see our agents in action!

In [ ]:
# Initialize the service
service = CulturalAgentService()

# Display available agents
print("🇧🇷 CIDADAO.AI AGENTS")
print("=" * 60)
for agent in service.list_agents():
    print(f"{agent['emoji']} {agent['name']}")
    print(f"   Role: {agent['role']}")
    print(f"   Expertise: {', '.join(agent['expertise'])}")
    print()

In [ ]:
# Test each agent with a relevant question
test_cases = [
    ("zumbi", "I found suspicious patterns in government contracts. There are multiple contracts awarded to the same company with values just below the bidding threshold.", "investigate"),
    ("anita", "Can you analyze the education spending trends over the last 5 years and identify any correlations with student performance?", "analyze"),
    ("drummond", "What does it mean when the government budget shows a 'contingency reserve'? Explain it simply.", "explain"),
    ("tiradentes", "I need a report on healthcare budget allocation in my municipality for a citizen oversight committee.", "report"),
    ("santos_dumont", "How does the agent routing system work in Cidadao.AI? I want to understand the architecture.", "explain")
]

print("🔬 TESTING AGENT RESPONSES")
print("=" * 70)

for agent_id, message, intent in test_cases:
    result = service.chat(agent_id, message, intent)
    
    print(f"\n{result['agent_emoji']} {result['agent_name']} ({result['agent_role']})")
    print(f"Question: {message[:60]}...")
    print("-" * 70)
    print(result['response'])
    print("=" * 70)

---

## Part 9: Specialized Investigation

In [ ]:
# Run a specialized investigation with Zumbi
investigation_data = """
MUNICIPAL CONTRACTS DATASET SUMMARY:
- 47 contracts awarded to "ABC Services Ltd" in 2023
- 43 of these contracts are valued between R$145,000 and R$149,900
- Legal bidding threshold is R$150,000
- ABC Services was registered 2 months before first contract
- Company address matches a residential building
- Same company won contracts in 12 different departments
"""

print("🔍 FRAUD INVESTIGATION ANALYSIS")
print("=" * 60)

investigation = service.investigate(
    data_description=investigation_data,
    analysis_type="anomaly"
)

print(f"\n{investigation['agent_emoji']} Analysis by {investigation['agent_name']}")
print(f"\n📋 FINDINGS:")
print(investigation['findings'])
print(f"\n⚠️ RISK LEVEL: {investigation['risk_level']}")
print(f"\n💡 RECOMMENDATIONS:")
print(investigation['recommendations'])
print(f"\n📊 Confidence: {investigation['confidence']:.1%}")

---

## Part 10: Report Generation

In [ ]:
# Generate a citizen-friendly report with Tiradentes
findings_summary = """
Key findings from healthcare budget analysis:
1. 23% of allocated funds were not spent
2. Emergency services received 15% less than previous year
3. Administrative costs increased by 40%
4. Rural health posts received only 8% of total budget
5. Medical equipment purchases were 60% below target
"""

print("📝 CITIZEN REPORT GENERATION")
print("=" * 60)

report = service.generate_report(
    topic="Municipal Healthcare Budget 2023",
    findings=findings_summary,
    audience="citizen"
)

print(f"\n{report['agent_emoji']} Report by {report['agent_name']}")
print(f"\n📄 {report['title']}")
print(f"\n📌 EXECUTIVE SUMMARY:")
print(report['executive_summary'])
print(f"\n📖 DETAILED REPORT:")
print(report['detailed_report'])
print(f"\n✅ ACTION ITEMS:")
print(report['action_items'])

---

## Part 11: Intelligent Agent Routing

The router automatically selects the best agent for each query based on intent and expertise.

In [ ]:
class MultiAgentOrchestrator:
    """
    Orchestrates multiple agents, routing queries to the most appropriate one.
    Can also chain agents for complex multi-step tasks.
    """
    
    def __init__(self):
        self.service = CulturalAgentService()
        self.router = dspy.ChainOfThought(IntentRouter)
        
        # Build agent descriptions for routing
        self.agent_descriptions = "\n".join([
            f"- {info.id}: {info.role}. Expertise: {', '.join(info.expertise)}"
            for info in AGENT_REGISTRY.values()
        ])
    
    def process(self, message: str, context: str = "") -> dict[str, Any]:
        """Process a message by routing to the best agent."""
        
        # Route to best agent
        routing = self.router(
            user_message=message,
            available_agents=self.agent_descriptions
        )
        
        # Get response from selected agent
        response = self.service.chat(
            agent_id=routing.selected_agent,
            message=message,
            intent_type=routing.intent_type,
            context=context
        )
        
        return {
            **response,
            "routing": {
                "selected_agent": routing.selected_agent,
                "intent_type": routing.intent_type,
                "reasoning": routing.reasoning,
                "confidence": routing.confidence
            }
        }
    
    def collaborative_investigation(self, topic: str) -> dict[str, Any]:
        """
        Run a multi-agent collaborative investigation.
        Chains: Zumbi (investigate) -> Anita (analyze) -> Tiradentes (report)
        """
        results = {}
        
        # Step 1: Zumbi investigates
        print("🔍 Step 1: Zumbi investigating...")
        investigation = self.service.chat(
            "zumbi", 
            f"Investigate this topic thoroughly: {topic}",
            "investigate"
        )
        results['investigation'] = investigation
        
        # Step 2: Anita analyzes findings
        print("📊 Step 2: Anita analyzing findings...")
        analysis = self.service.chat(
            "anita",
            f"Analyze these investigation findings and identify patterns: {investigation['response']}",
            "analyze"
        )
        results['analysis'] = analysis
        
        # Step 3: Tiradentes creates report
        print("📝 Step 3: Tiradentes generating report...")
        report = self.service.generate_report(
            topic=topic,
            findings=f"Investigation: {investigation['response']}\n\nAnalysis: {analysis['response']}",
            audience="citizen"
        )
        results['report'] = report
        
        return results

In [ ]:
# Test the orchestrator
orchestrator = MultiAgentOrchestrator()

test_queries = [
    "I need to investigate potential fraud in public contracts",
    "What are the spending trends in education over the last 5 years?",
    "Generate a report on healthcare budget for citizens",
    "Explain what a 'contingency fund' means in simple terms",
    "How does the data pipeline architecture work in this system?"
]

print("🤖 INTELLIGENT AGENT ROUTING")
print("=" * 70)

for query in test_queries:
    result = orchestrator.process(query)
    routing = result['routing']
    
    print(f"\n📨 Query: {query}")
    print(f"   🎯 Routed to: {result['agent_emoji']} {result['agent_name']}")
    print(f"   💭 Reasoning: {routing['reasoning']}")
    print(f"   📊 Confidence: {routing['confidence']:.1%}")
    print("-" * 70)

---

## Part 12: Multi-Agent Collaboration

In [ ]:
# Run a collaborative investigation
print("🤝 COLLABORATIVE MULTI-AGENT INVESTIGATION")
print("=" * 70)
print("Topic: Suspicious patterns in municipal construction contracts\n")

collab_results = orchestrator.collaborative_investigation(
    "Suspicious patterns in municipal construction contracts where the same companies keep winning bids"
)

print("\n" + "=" * 70)
print("📋 FINAL COLLABORATIVE REPORT")
print("=" * 70)

print(f"\n🔍 INVESTIGATION by {collab_results['investigation']['agent_name']}:")
print(collab_results['investigation']['response'][:500] + "...")

print(f"\n📊 ANALYSIS by {collab_results['analysis']['agent_name']}:")
print(collab_results['analysis']['response'][:500] + "...")

print(f"\n📝 CITIZEN REPORT by {collab_results['report']['agent_name']}:")
print(f"Title: {collab_results['report']['title']}")
print(f"\nSummary: {collab_results['report']['executive_summary']}")

---

## Part 13: Streaming Responses

For better UX, implement streaming responses that show text as it's generated.

In [ ]:
def stream_response(service: CulturalAgentService, agent_id: str, message: str) -> Generator[dict, None, None]:
    """
    Stream agent response in chunks for real-time display.
    Note: This simulates streaming by chunking the full response.
    For true streaming, use dspy.streamify() with supported models.
    """
    # Get full response
    result = service.chat(agent_id, message)
    response_text = result['response']
    
    # Yield start event
    yield {
        "type": "start",
        "agent_id": result['agent_id'],
        "agent_name": result['agent_name'],
        "agent_emoji": result['agent_emoji']
    }
    
    # Stream in word chunks
    words = response_text.split()
    chunk = ""
    
    for i, word in enumerate(words):
        chunk += word + " "
        if (i + 1) % 5 == 0:  # Yield every 5 words
            yield {
                "type": "chunk",
                "content": chunk.strip()
            }
            chunk = ""
    
    # Yield remaining
    if chunk.strip():
        yield {"type": "chunk", "content": chunk.strip()}
    
    # Yield complete event
    yield {
        "type": "complete",
        "success": result['success']
    }


# Demo streaming
print("📡 STREAMING RESPONSE DEMO")
print("=" * 60)

for event in stream_response(service, "drummond", "What is transparency in government?"):
    if event['type'] == 'start':
        print(f"\n{event['agent_emoji']} {event['agent_name']} is typing...\n")
    elif event['type'] == 'chunk':
        print(event['content'], end=' ', flush=True)
    elif event['type'] == 'complete':
        print("\n\n✓ Response complete")

---

## Part 14: Optimization with DSPy

Use DSPy optimizers to improve agent responses based on examples and metrics.

In [ ]:
# Create training examples for optimization
training_examples = [
    dspy.Example(
        user_message="I found contracts just below the bidding threshold",
        intent_type="investigate",
        expected_elements=["fraud", "bidding", "investigate", "threshold"]
    ).with_inputs("user_message", "intent_type"),
    
    dspy.Example(
        user_message="Show me education spending trends",
        intent_type="analyze",
        expected_elements=["trend", "analysis", "data", "pattern"]
    ).with_inputs("user_message", "intent_type"),
    
    dspy.Example(
        user_message="Explain the budget in simple terms",
        intent_type="explain",
        expected_elements=["simple", "means", "understand", "citizen"]
    ).with_inputs("user_message", "intent_type"),
]

def response_quality_metric(example, prediction, trace=None):
    """Metric to evaluate response quality."""
    response = prediction.response.lower()
    
    # Check if response contains expected elements
    matches = sum(1 for elem in example.expected_elements if elem in response)
    score = matches / len(example.expected_elements)
    
    # Bonus for personality markers
    personality_bonus = 0.1 if any(marker in response for marker in 
        ["zumbi", "anita", "drummond", "tiradentes", "santos", "investigate", "analyze"]) else 0
    
    return min(1.0, score + personality_bonus)

print("📈 Response quality metric defined")
print("   Use with dspy.MIPROv2 or dspy.BootstrapFewShot for optimization")

In [ ]:
# Example of optimization setup (uncomment to run - requires API calls)
"""
from dspy.teleprompt import BootstrapFewShot

# Create optimizer
optimizer = BootstrapFewShot(
    metric=response_quality_metric,
    max_bootstrapped_demos=3,
    max_labeled_demos=3
)

# Optimize the chat module
optimized_chat = optimizer.compile(
    CulturalAgentChat(),
    trainset=training_examples
)

# Save optimized module
optimized_chat.save("optimized_cultural_agents.json")
print("✓ Optimized agents saved!")
"""
print("💡 Optimization code ready - uncomment to run with your training data")

---

## Part 15: Saving and Loading

Save your agents for production use and load them later.

In [ ]:
# Save the modules
service.chat_module.save("cultural_agents_chat.json")
service.investigator_module.save("cultural_agents_investigator.json")
service.reporter_module.save("cultural_agents_reporter.json")

print("✓ All modules saved!")
print("  - cultural_agents_chat.json")
print("  - cultural_agents_investigator.json")
print("  - cultural_agents_reporter.json")

In [ ]:
# Load and verify
loaded_chat = CulturalAgentChat()
loaded_chat.load("cultural_agents_chat.json")

# Test loaded module
result = loaded_chat(
    agent_personality=AgentPersonality.SANTOS_DUMONT,
    user_message="How do I get started with DSPy?",
    intent_type="explain"
)

print("✓ Loaded module test:")
print(f"Response preview: {result.response[:200]}...")

---

## Summary

In this comprehensive tutorial, you learned how to:

1. **Design Agent Architecture**: Create a scalable multi-agent system with cultural personalities
2. **Define DSPy Signatures**: Structure input/output contracts for different agent capabilities
3. **Build Custom Modules**: Combine signatures with reasoning strategies like ChainOfThought
4. **Create a Service Layer**: Provide a clean API with fallback handling
5. **Implement Agent Routing**: Automatically direct queries to the best agent
6. **Enable Collaboration**: Chain multiple agents for complex investigations
7. **Add Streaming**: Improve UX with real-time response generation
8. **Optimize Performance**: Use DSPy optimizers to improve agent quality

## Resources

- [DSPy Documentation](https://dspy.ai/)
- [Cidadao.AI Live Demo](https://cidadao-ai-frontend.vercel.app/) - See all 22 agents in action
- [Cidadao.AI Backend](https://github.com/anderson-ufrj/cidadao.ai-backend) - Full production implementation
- [DSPy Agents Tutorial](../agents/index.ipynb)
- [DSPy ReAct Documentation](../../api/modules/ReAct.md)

## About Cidadao.AI

Cidadao.AI is a Brazilian government transparency platform created by **Anderson Henrique da Silva** as a thesis project at **Instituto Federal do Sul de Minas Gerais (IFSULDEMINAS)**, supervised by Professor **Aracele Garcia de Oliveira Fassbinder**.

The platform uses DSPy to power 22 AI agents, each with a unique Brazilian cultural identity, to help citizens analyze public data and fight corruption. It demonstrates how AI can be made culturally relevant and accessible to all citizens.

---

*"Knowledge must be free, like my inventions."* - Santos-Dumont